In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import itertools
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from xgboost import XGBRegressor

from sklearn.multioutput import MultiOutputRegressor

In [ ]:
df_Uber = pd.read_excel('/content/uber_hackathon_v2_mock_data.xlsx')

In [ ]:
df_rider = pd.read_excel("/content/uber_hackathon_v2_mock_data.xlsx", sheet_name="riders")

In [ ]:
df_customer = pd.read_excel("/content/uber_hackathon_v2_mock_data.xlsx", sheet_name="customers")

In [ ]:
df_merchants = pd.read_excel("/content/uber_hackathon_v2_mock_data.xlsx", sheet_name="merchants")

In [ ]:
df_merchants = pd.read_excel("/content/uber_hackathon_v2_mock_data.xlsx", sheet_name="merchants")

In [ ]:
df_rides_trips = pd.read_excel("/content/uber_hackathon_v2_mock_data.xlsx", sheet_name="rides_trips")

In [ ]:
df_eats_orders = pd.read_excel("/content/uber_hackathon_v2_mock_data.xlsx", sheet_name="eats_orders")

In [ ]:
df_jobs_like= pd.read_excel("/content/uber_hackathon_v2_mock_data.xlsx", sheet_name="jobs_like")

In [ ]:
df_earnering_daily= pd.read_excel("/content/uber_hackathon_v2_mock_data.xlsx", sheet_name="earnings_daily")

In [ ]:
df_incentives_weekly= pd.read_excel("/content/uber_hackathon_v2_mock_data.xlsx", sheet_name="incentives_weekly")

In [ ]:
df_surge_by_hour= pd.read_excel("/content/uber_hackathon_v2_mock_data.xlsx", sheet_name="surge_by_hour")

In [ ]:
df_cancellation_rates= pd.read_excel("/content/uber_hackathon_v2_mock_data.xlsx", sheet_name="cancellation_rates")

In [ ]:
df_weather_daily = pd.read_excel("/content/uber_hackathon_v2_mock_data.xlsx", sheet_name="weather_daily")

## Machine Learning

In [ ]:
# Clusters

In [ ]:
kmeans = KMeans(n_clusters=6, random_state=42)
cluster_df = df_rides_trips[['rider_id', 'city_id', 'pickup_lat', 'pickup_lon', 'start_time', 'date', 'net_earnings', 'drop_lat', 'drop_lon']]
cluster_df['cluster'] = kmeans.fit_predict(cluster_df[['pickup_lat', 'pickup_lon']])
centers = pd.DataFrame(kmeans.cluster_centers_, columns=['center_latitude', 'center_longitude'])
centers['cluster'] = centers.index

/tmp/ipython-input-3916292153.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_df['cluster'] = kmeans.fit_predict(cluster_df[['pickup_lat', 'pickup_lon']])


In [ ]:
centers

,center_latitude,center_longitude,cluster
0,51.919442,4.478279,0
1,52.085142,5.131674,1
2,51.431682,5.469674,2
3,52.362816,4.851503,3
4,52.067159,4.296510,4
5,52.364455,4.921073,5


## Rider Departure Data

In [ ]:
rider_city_data = df_rides_trips[['rider_id', 'city_id', 'pickup_lat', 'pickup_lon', 'start_time', 'date', 'net_earnings']]
rider_city_data = pd.merge(rider_city_data, df_weather_daily, on=['date', 'city_id'], how='left')

In [ ]:
city_1_data = rider_city_data[rider_city_data['city_id'] == 1].drop(columns=['city_id'])

In [ ]:
city_1_data['day'] = pd.to_datetime(city_1_data['date']).dt.weekday

In [ ]:
city_1_data['hour'] = pd.to_datetime(city_1_data['start_time']).dt.hour + 0.5 * (pd.to_datetime(city_1_data['start_time']).dt.minute // 30)

In [ ]:
city_1_data['cluster'] = kmeans.fit_predict(city_1_data[['pickup_lat', 'pickup_lon']])

In [ ]:
city_1_grouped = city_1_data[['weather', 'day', 'hour', 'cluster', 'rider_id']]

In [ ]:
city_1_grouped = city_1_grouped.groupby(['weather', 'day', 'hour',  'cluster']).agg(rides=('rider_id','count'))

In [ ]:
city_1_pivot = city_1_grouped.unstack('cluster', fill_value=0)
city_1_pivot.columns = city_1_pivot.columns.droplevel(0)  # remove 'rides'
city_1_pivot.columns.name = None  # remove the leftover name 'cluster'
city_1_pivot.reset_index(inplace=True)

In [ ]:
le = LabelEncoder()
city_1_pivot['weather_enc'] = le.fit_transform(city_1_pivot['weather'])

# Features
X = city_1_pivot[['weather_enc','day','hour']]
# Multi-output targets (one column per cluster)
y = city_1_pivot[[0,1,2,3,4,5]]

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Base model
xgb = XGBRegressor(objective='count:poisson', n_estimators=200, random_state=42)

# Wrap for multi-output
multi_xgb = MultiOutputRegressor(xgb)
multi_xgb.fit(X_train, y_train)

# Predict
y_pred = multi_xgb.predict(X_test)

In [ ]:
for i, col in enumerate(y.columns):
    r2 = r2_score(y_test[col], y_pred[:,i])
    print(f"Cluster {col} R^2: {r2:.2f}")

Cluster 0 R^2: -0.08
Cluster 1 R^2: -0.64
Cluster 2 R^2: -0.94
Cluster 3 R^2: -2.96
Cluster 4 R^2: -0.56
Cluster 5 R^2: -3.81


In [ ]:
city_1_data.head()

,rider_id,pickup_lat,pickup_lon,start_time,date,net_earnings,weather,day,hour,cluster
5,R2351,52.36298,4.90663,2023-04-08 07:54:00,2023-04-08,8.66,clear,5,7.5,5
8,R2341,52.38348,4.86060,2023-01-17 11:54:00,2023-01-17,4.52,clear,1,11.5,1
12,R2095,52.34498,4.84592,2023-01-22 08:59:00,2023-01-22,13.55,clear,6,8.5,2
16,R2269,52.33970,4.84998,2023-03-04 14:32:00,2023-03-04,2.12,rain,5,14.5,2
20,R2232,52.38530,4.86020,2023-03-06 15:42:00,2023-03-06,8.11,rain,0,15.5,1


In [ ]:
city_1_data.to_excel('raw_data.xlsx')

## Driver Arrival Data

In [ ]:
driver_city_data = df_rides_trips[['driver_id', 'city_id', 'end_time', 'date','drop_lat', 'drop_lon', 'net_earnings']]
driver_city_data = pd.merge(driver_city_data, df_weather_daily, on=['date', 'city_id'], how='left')

In [ ]:
city_1_driver = driver_city_data[driver_city_data['city_id'] == 1].drop(columns=['city_id'])
city_1_driver['day'] = pd.to_datetime(city_1_driver['date']).dt.weekday
city_1_driver['hour'] = pd.to_datetime(city_1_driver['end_time']).dt.hour + 0.5 * (pd.to_datetime(city_1_driver['end_time']).dt.minute // 30)

city_1_driver['cluster'] = kmeans.fit_predict(city_1_driver[['drop_lat', 'drop_lon']])
city_1_grouped_driver = city_1_driver[['weather', 'day', 'hour', 'cluster', 'driver_id']]
city_1_grouped_driver = city_1_grouped_driver.groupby(['weather', 'day', 'hour',  'cluster']).agg(rides=('driver_id','count'))
city_1_pivot_driver = city_1_grouped_driver.unstack('cluster', fill_value=0)
city_1_pivot_driver.columns = city_1_pivot_driver.columns.droplevel(0)  # remove 'rides'
city_1_pivot_driver.columns.name = None  # remove the leftover name 'cluster'
city_1_pivot_driver.reset_index(inplace=True)

le = LabelEncoder()
city_1_pivot_driver['weather_enc'] = le.fit_transform(city_1_pivot_driver['weather'])

# Features
X_driver = city_1_pivot_driver[['weather_enc','day','hour']]
# Multi-output targets (one column per cluster)
y_driver = city_1_pivot_driver[[0,1,2,3,4,5]]

# Split
X_train_driver, X_test_driver, y_train_driver, y_test_driver = train_test_split(X_driver, y_driver, test_size=0.2, random_state=42)

# Base model
xgb_driver = XGBRegressor(objective='count:poisson', n_estimators=200, random_state=42)

# Wrap for multi-output
multi_xgb_driver = MultiOutputRegressor(xgb_driver)
multi_xgb_driver.fit(X_train_driver, y_train_driver)

# Predict
y_pred_driver = multi_xgb_driver.predict(X_test_driver)

for i, col in enumerate(y.columns):
    r2 = r2_score(y_test_driver[col], y_pred_driver[:,i])
    print(f"Cluster {col} R^2: {r2:.2f}")

Cluster 0 R^2: -0.16
Cluster 1 R^2: -0.44
Cluster 2 R^2: -0.45
Cluster 3 R^2: -0.68
Cluster 4 R^2: -0.43
Cluster 5 R^2: -0.40


In [ ]:
city_1_driver.head()

,driver_id,end_time,date,drop_lat,drop_lon,net_earnings,weather,day,hour,cluster
5,E10131,2023-04-08 08:06:00,2023-04-08,52.33967,4.84363,8.66,clear,5,8.0,3
8,E10020,2023-01-17 12:04:00,2023-01-17,52.33935,4.84197,4.52,clear,1,12.0,3
12,E10058,2023-01-22 09:31:00,2023-01-22,52.38706,4.85387,13.55,clear,6,9.5,2
16,E10027,2023-03-04 15:03:00,2023-03-04,52.36684,4.93222,2.12,rain,5,15.0,1
20,E10063,2023-03-06 15:53:00,2023-03-06,52.35877,4.90065,8.11,rain,0,15.5,5


## Predictions

In [ ]:
weathers = ['clear', 'rain', 'snow']
hours = np.arange(0, 24, 0.5)  # 0, 0.5, ..., 23.5
days = np.arange(0, 7)  # 0,1,2,...,6

# Generate all combinations
combinations = list(itertools.product(weathers, days, hours))

# Convert to DataFrame
df_all_inputs = pd.DataFrame(combinations, columns=['weather', 'day', 'hour'])

df_all_inputs['weather_enc'] = le.transform(df_all_inputs['weather'])

X_all = df_all_inputs[['weather_enc', 'day', 'hour']]

## Rider Departure Data

In [ ]:
y_all_pred = multi_xgb.predict(X_all)

# Optional: round predictions to integers
y_all_pred_rounded = np.clip(np.round(y_all_pred), 0, None).astype(int)

# Combine with input DataFrame
df_predictions = pd.concat([df_all_inputs.reset_index(drop=True),
                            pd.DataFrame(y_all_pred_rounded, columns=[0,1,2,3,4,5])], axis=1)

df_predictions['weather'] = le.inverse_transform(df_predictions['weather_enc'])
df_predictions.drop(columns=['weather_enc'], inplace=True)

In [ ]:
df_predictions

,weather,day,hour,0,1,2,3,4,5
0,clear,0,0.0,0,0,0,0,0,0
1,clear,0,0.5,0,0,1,0,0,1
2,clear,0,1.0,1,0,0,0,0,0
3,clear,0,1.5,0,1,0,0,0,0
4,clear,0,2.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1003,snow,6,21.5,0,1,0,0,1,0
1004,snow,6,22.0,0,1,0,0,0,0
1005,snow,6,22.5,0,0,0,0,0,0
1006,snow,6,23.0,0,0,0,0,0,0


## Driver Arrival Data

In [ ]:
y_all_pred_driver = multi_xgb_driver.predict(X_all)

# Optional: round predictions to integers
y_all_pred_rounded_driver = np.clip(np.round(y_all_pred_driver), 0, None).astype(int)

# Combine with input DataFrame
df_predictions_driver = pd.concat([df_all_inputs.reset_index(drop=True),
                            pd.DataFrame(y_all_pred_rounded_driver, columns=[0,1,2,3,4,5])], axis=1)


In [ ]:
df_predictions_driver['weather'] = le.inverse_transform(df_predictions_driver['weather_enc'])
df_predictions_driver.drop(columns=['weather_enc'], inplace=True)

In [ ]:
df_predictions_driver

,weather,day,hour,0,1,2,3,4,5
0,clear,0,0.0,0,1,0,0,0,2
1,clear,0,0.5,0,1,0,0,0,1
2,clear,0,1.0,0,1,0,0,0,1
3,clear,0,1.5,1,0,0,0,1,0
4,clear,0,2.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1003,snow,6,21.5,1,0,0,0,0,2
1004,snow,6,22.0,0,0,0,0,0,0
1005,snow,6,22.5,0,0,0,0,0,0
1006,snow,6,23.0,0,0,0,0,0,0


## Rider to Driver Ratio

In [ ]:
['weather', 'day', 'hour', 0, 1, 2, 3, 4, 5]

rider_driver_merged = pd.merge(
    df_predictions,
    df_predictions_driver,
    on=['weather', 'day', 'hour'],
    suffixes=('_rider', '_driver')
)

In [ ]:
rider_driver_merged

,weather,day,hour,0_rider,1_rider,2_rider,3_rider,4_rider,5_rider,0_driver,1_driver,2_driver,3_driver,4_driver,5_driver
0,clear,0,0.0,0,0,0,0,0,0,0,1,0,0,0,2
1,clear,0,0.5,0,0,1,0,0,1,0,1,0,0,0,1
2,clear,0,1.0,1,0,0,0,0,0,0,1,0,0,0,1
3,clear,0,1.5,0,1,0,0,0,0,1,0,0,0,1,0
4,clear,0,2.0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,snow,6,21.5,0,1,0,0,1,0,1,0,0,0,0,2
1004,snow,6,22.0,0,1,0,0,0,0,0,0,0,0,0,0
1005,snow,6,22.5,0,0,0,0,0,0,0,0,0,0,0,0
1006,snow,6,23.0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Add 1 to the denominator
for c in [0, 1, 2, 3, 4, 5]:
    rider_driver_merged[f'{c}_riders_per_driver'] = rider_driver_merged[f'{c}_rider'] / (rider_driver_merged[f'{c}_driver'] + 1)

In [ ]:
rider_driver_merged

,weather,day,hour,0_rider,1_rider,2_rider,3_rider,4_rider,5_rider,0_driver,...,2_driver,3_driver,4_driver,5_driver,0_riders_per_driver,1_riders_per_driver,2_riders_per_driver,3_riders_per_driver,4_riders_per_driver,5_riders_per_driver
0,clear,0,0.0,0,0,0,0,0,0,0,...,0,0,0,2,0.0,0.0,0.0,0.0,0.0,0.0
1,clear,0,0.5,0,0,1,0,0,1,0,...,0,0,0,1,0.0,0.0,1.0,0.0,0.0,0.5
2,clear,0,1.0,1,0,0,0,0,0,0,...,0,0,0,1,1.0,0.0,0.0,0.0,0.0,0.0
3,clear,0,1.5,0,1,0,0,0,0,1,...,0,0,1,0,0.0,1.0,0.0,0.0,0.0,0.0
4,clear,0,2.0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,snow,6,21.5,0,1,0,0,1,0,1,...,0,0,0,2,0.0,1.0,0.0,0.0,1.0,0.0
1004,snow,6,22.0,0,1,0,0,0,0,0,...,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0
1005,snow,6,22.5,0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1006,snow,6,23.0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
for c in [0, 1, 2, 3, 4, 5]:
  rider_driver_merged = rider_driver_merged.drop(columns=[f'{c}_rider'])
  rider_driver_merged = rider_driver_merged.drop(columns=[f'{c}_driver'])
  rider_driver_merged = rider_driver_merged.rename(columns={f'{c}_riders_per_driver': c})

rider_driver_merged

,weather,day,hour,0,1,2,3,4,5
0,clear,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,clear,0,0.5,0.0,0.0,1.0,0.0,0.0,0.5
2,clear,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,clear,0,1.5,0.0,1.0,0.0,0.0,0.0,0.0
4,clear,0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
1003,snow,6,21.5,0.0,1.0,0.0,0.0,1.0,0.0
1004,snow,6,22.0,0.0,1.0,0.0,0.0,0.0,0.0
1005,snow,6,22.5,0.0,0.0,0.0,0.0,0.0,0.0
1006,snow,6,23.0,0.0,0.0,0.0,0.0,0.0,0.0


## Edges of City Neighbourhood Graph

In [ ]:
edges_raw_df = pd.read_csv('fullgrid_probabilities_expected_earnings.csv')

In [ ]:
edges_raw_df = edges_raw_df.rename(columns={'smoothed_mean_net_earning_lambda20': 'expected_earnings'})

In [ ]:
edges_raw_df

,cluster,day,hour,weather,cluster_drop,probability_laplace,n_observed,expected_earnings,expected_net_earning_smoothed
0,0,0,0.0,clear,0,0.000000,0,8.892934,0.000000
1,0,0,0.0,clear,1,0.166667,0,8.892934,1.482156
2,0,0,0.0,clear,2,0.166667,0,8.892934,1.482156
3,0,0,0.0,clear,3,0.333333,1,8.828508,2.942836
4,0,0,0.0,clear,4,0.166667,0,8.892934,1.482156
...,...,...,...,...,...,...,...,...,...
36283,5,6,23.5,snow,1,0.200000,0,8.892934,1.778587
36284,5,6,23.5,snow,2,0.200000,0,8.892934,1.778587
36285,5,6,23.5,snow,3,0.200000,0,8.892934,1.778587
36286,5,6,23.5,snow,4,0.200000,0,8.892934,1.778587


In [ ]:
edges_raw_df['expected_earnings'] = edges_raw_df['expected_earnings'].bfill()

In [ ]:
edges_earnings = edges_raw_df[['cluster', 'day', 'hour', 'weather', 'expected_earnings']]
edges_earnings = edges_earnings.drop_duplicates(subset=['cluster', 'day', 'hour', 'weather'])
edges_earnings

,cluster,day,hour,weather,expected_earnings
0,0,0,0.0,clear,8.892934
6,0,0,0.0,rain,8.892934
12,0,0,0.0,snow,8.892934
18,0,0,0.5,clear,8.892934
24,0,0,0.5,rain,8.892934
...,...,...,...,...,...
36258,5,6,23.0,rain,8.892934
36264,5,6,23.0,snow,8.892934
36270,5,6,23.5,clear,8.892934
36276,5,6,23.5,rain,8.892934


In [ ]:
edges_earnings = (
    edges_earnings.sort_values(by=['weather', 'day', 'hour'], ascending=[True, True, True])
      .reset_index(drop=True)
)

edges_earnings

,cluster,day,hour,weather,expected_earnings
0,0,0,0.0,clear,8.892934
1,1,0,0.0,clear,8.892934
2,2,0,0.0,clear,8.892934
3,3,0,0.0,clear,8.892934
4,4,0,0.0,clear,8.892934
...,...,...,...,...,...
6043,1,6,23.5,snow,8.892934
6044,2,6,23.5,snow,8.892934
6045,3,6,23.5,snow,8.892934
6046,4,6,23.5,snow,8.892934


In [ ]:
earnings_pivot = (
    edges_earnings.pivot_table(
        index=['weather', 'day', 'hour'],     # these stay as rows
        columns='cluster',                    # this becomes new columns
        values='expected_earnings',           # fill cells with this
        fill_value=0                          # optional: replace NaN with 0
    )
    .reset_index()                            # make 'weather', 'day', 'hour' normal columns
)
earnings_pivot

cluster,weather,day,hour,0,1,2,3,4,5
0,clear,0,0.0,8.892934,8.892934,8.892934,8.892934,8.892934,8.892934
1,clear,0,0.5,8.892934,8.892934,8.892934,8.892934,8.892934,8.892934
2,clear,0,1.0,9.007556,8.892934,8.892934,8.892934,8.892934,8.892934
3,clear,0,1.5,8.892934,8.892934,8.892934,8.892934,8.892934,8.892934
4,clear,0,2.0,8.892934,8.892934,8.892934,8.892934,8.892934,8.892934
...,...,...,...,...,...,...,...,...,...
1003,snow,6,21.5,8.892934,8.892934,8.892934,8.892934,8.892934,8.892934
1004,snow,6,22.0,8.892934,8.892934,8.892934,8.892934,8.892934,8.892934
1005,snow,6,22.5,8.892934,8.892934,8.892934,8.892934,8.892934,8.892934
1006,snow,6,23.0,8.892934,8.892934,8.892934,8.892934,8.892934,8.892934


## Export Data

In [ ]:
df_predictions_per_driver = (rider_driver_merged.set_index(['weather','day','hour']) *
                 earnings_pivot.set_index(['weather','day','hour'])).reset_index()
df_predictions_per_driver

,weather,day,hour,0,1,2,3,4,5
0,clear,0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,clear,0,0.5,0.000000,0.000000,8.892934,0.0,0.000000,4.446467
2,clear,0,1.0,9.007556,0.000000,0.000000,0.0,0.000000,0.000000
3,clear,0,1.5,0.000000,8.892934,0.000000,0.0,0.000000,0.000000
4,clear,0,2.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
1003,snow,6,21.5,0.000000,8.892934,0.000000,0.0,8.892934,0.000000
1004,snow,6,22.0,0.000000,8.892934,0.000000,0.0,0.000000,0.000000
1005,snow,6,22.5,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1006,snow,6,23.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [ ]:
# Export as json:
df_predictions_per_driver.to_json('expected_earnings_per_driver.json', orient='split', compression='infer', index=True)

In [ ]:
df_predictions_per_driver.to_excel('recommendation system based on revenue.xlsx')